# Alle notwendigen Inputdaten einlesen

## Imports & Generische Konfig

In [1]:
#Variables
import config

#Preprocessing
import preprocessing_functions as pf

#Libraries
import requests
import json
import pandas as pd
import glob
from bs4 import BeautifulSoup
from sqlalchemy import text
from datetime import date

#Verbindung zur DB aufbauen mit preprocessing function
pf.db_connect(config.db_name)


## Historische Feinstaub Daten (ZH) auslesen

In [3]:
url = 'https://data.stadt-zuerich.ch/dataset/ugz_luftschadstoffmessung_tageswerte'

# Verbindung zur Webseite herstellen
response = requests.get(url)

# Webseite parsen
soup = BeautifulSoup(response.text, 'html.parser')

# Alle CSV-Links finden
csv_links = []
for a in soup.find_all('a', href=True):
    link = a['href']
    if 'ugz_ogd_air_d1_2' in link and link.endswith('.csv'):
        csv_links.append(link)

# CSV-Dateien herunterladen
for link in csv_links:
    response = requests.get(link)
    with open('csv-files/'+link.split('/')[-1], 'wb') as f:
        f.write(response.content)

# Liste aller ugz CSV-Dateien im Verzeichnis csv-files
csv_files = glob.glob('csv-files/ugz*.csv')

# Leeres DataFrame zum Speichern der gefilterten Daten erstellen
AQI_history_ZH = pd.DataFrame()
    
# Durch alle CSV-Dateien iterieren und Daten filtern
for file in csv_files:
    df = pd.read_csv(file)
    df_filtered = df[df['Parameter'] == 'PM10']
    df_filtered = df_filtered[df_filtered['Standort'] == 'Zch_Stampfenbachstrasse']
    df_filtered['Datum'] = df_filtered['Datum'].apply(pf.format_timestamp)
    AQI_history_ZH = pd.concat([AQI_history_ZH, df_filtered])

# Index neu setzen
AQI_history_ZH.reset_index(drop=True, inplace=True)

# Dataframe in die DB schreiben
AQI_history_ZH.to_sql(config.db_AQI_history, config.db_login, if_exists='append', index=False)


168

## Historische Wetterdaten (ZH) aus csv einlesen

In [ ]:
# CSV-File ins Dataframe einlesen
weather_history_ZH = pd.read_csv(config.csv_weatherhistory)

weather_history_ZH['datetime'] = weather_history_ZH['datetime'].apply(pf.format_timestamp)


# Index neu setzen
weather_history_ZH.reset_index(drop=True, inplace=True)

# Dataframe in die DB schreiben
weather_history_ZH.to_sql(config.db_weather_history, config.db_login, if_exists='append', index=False)


## DB Testen für eingelesene Daten

In [7]:
# Check Output von DB für AQI_history
SQLquery = text('SELECT * FROM ' + config.db_AQI_history + ' AS AQI_History')
df_AQIHistoryDB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_AQIHistoryDB.head()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
0,04.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,22.84,bereinigt
1,05.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,22.63,bereinigt
2,06.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,10.93,bereinigt
3,07.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,NaN,bereinigt
4,08.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,18.01,bereinigt


In [8]:
# Check Output von DB für weather_history
SQLquery = text('SELECT * FROM ' + config.db_weather_history + ' AS weather_History')
df_weatherHistory = pd.read_sql(SQLquery, con=config.db_login.connect())

df_weatherHistory.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,Zch_Stampfenbachstrasse,2001-01-01,1.3,-5.9,-1.8,1.3,-5.9,-2.2,-4.5,81.9,...,NaN,NaN,NaN,2001-01-01T08:13:21,2001-01-01T16:45:47,0.22,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"06689099999,06670099999,06660099999"
1,Zch_Stampfenbachstrasse,2001-01-02,5.6,0.9,3.4,5.3,-1.7,2.1,2.0,90.1,...,NaN,NaN,NaN,2001-01-02T08:13:19,2001-01-02T16:46:46,0.25,"Snow, Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,"06689099999,06670099999,06660099999"
2,Zch_Stampfenbachstrasse,2001-01-03,7.2,2.1,5.1,4.6,0.8,2.6,1.7,78.7,...,NaN,NaN,NaN,2001-01-03T08:13:14,2001-01-03T16:47:46,0.29,"Rain, Partially cloudy",Partly cloudy throughout the day with early mo...,rain,"06689099999,06670099999,06660099999"
3,Zch_Stampfenbachstrasse,2001-01-04,6.6,0.1,3.3,4.0,0.1,2.1,0.5,82.0,...,NaN,NaN,NaN,2001-01-04T08:13:07,2001-01-04T16:48:49,0.32,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"06689099999,06670099999,06660099999"
4,Zch_Stampfenbachstrasse,2001-01-05,7.3,1.1,4.3,6.4,1.1,3.9,1.9,84.8,...,NaN,NaN,NaN,2001-01-05T08:12:57,2001-01-05T16:49:54,0.35,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"06689099999,06670099999,06660099999"


In [11]:
#Gemeinsame Abfrage der Tabellen
SQLquery = text('SELECT * FROM ' + config.db_weather_history + ' AS w JOIN ' + config.db_AQI_history + ' AS aqi ON w."datetime" = aqi."Datum"')
df_METEODB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_METEODB.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,description,icon,stations,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
0,Zch_Stampfenbachstrasse,04.01.01,6.6,0.1,3.3,4.0,0.1,2.1,0.5,82.0,...,Partly cloudy throughout the day with late aft...,rain,"06689099999,06670099999,06660099999",04.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,22.84,bereinigt
1,Zch_Stampfenbachstrasse,05.01.01,7.3,1.1,4.3,6.4,1.1,3.9,1.9,84.8,...,Partly cloudy throughout the day with rain.,rain,"06689099999,06670099999,06660099999",05.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,22.63,bereinigt
2,Zch_Stampfenbachstrasse,06.01.01,8.2,3.4,6.2,7.9,2.0,4.9,4.9,91.9,...,Cloudy skies throughout the day with a chance ...,rain,"06689099999,06670099999,06660099999",06.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,10.93,bereinigt
3,Zch_Stampfenbachstrasse,07.01.01,2.4,1.1,1.7,2.1,-0.2,1.5,0.8,93.5,...,Cloudy skies throughout the day with a chance ...,rain,"06689099999,06670099999,06660099999",07.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,NaN,bereinigt
4,Zch_Stampfenbachstrasse,08.01.01,2.2,1.2,1.8,2.2,1.2,1.8,0.8,92.9,...,Cloudy skies throughout the day with a chance ...,rain,"06689099999,06670099999,06660099999",08.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,18.01,bereinigt
